In [0]:
from pyspark.sql.functions import * 
from pyspark.sql.types import *
from pyspark.sql.window import Window

# Silver Data Transformation (Title files)

In [0]:
df = spark.read.format("delta")\
        .option("header",True)\
        .option("inferSchema",True)\
        .load("abfss://01-bronze@project03storage.dfs.core.windows.net/netflix_titles")

In [0]:
df.limit(10).display()

duration_minutes,duration_seasons,type,title,date_added,release_year,rating,description,show_id,_rescued_data
90,null,Movie,Norm of the North: King Sized Adventure,9/9/2019,2019,TV-PG,"Before planning an awesome wedding for his grandfather, a polar bear king must take back a stolen artifact from an evil archaeologist first.",81145628,null
94,null,Movie,Jandino: Whatever it Takes,9/9/2016,2016,TV-MA,"""Jandino Asporaat riffs on the challenges of raising kids and serenades the audience with a rousing rendition of """"Sex on Fire"""" in his comedy show.""",80117401,null
null,1,TV Show,Transformers Prime,9/8/2018,2013,TV-Y7-FV,"With the help of three human allies, the Autobots once again protect Earth from the onslaught of the Decepticons and their leader, Megatron.",70234439,null
null,1,TV Show,Transformers: Robots in Disguise,9/8/2018,2016,TV-Y7,"When a prison ship crash unleashes hundreds of Decepticons on Earth, Bumblebee leads a new Autobot force to protect humankind.",80058654,null
99,null,Movie,#realityhigh,9/8/2017,2017,TV-14,"When nerdy high schooler Dani finally attracts the interest of her longtime crush, she lands in the cross hairs of his ex, a social media celebrity.",80125979,null
null,1,TV Show,Apaches,9/8/2017,2016,TV-MA,A young journalist is forced into a life of crime to save his father and family in this series based on the novel by Miguel Sáez Carral.,80163890,null
110,null,Movie,Automata,9/8/2017,2014,R,"In a dystopian future, an insurance adjuster for a tech company investigates a robot killed for violating protocol and discovers a global conspiracy.",70304989,null
60,null,Movie,Fabrizio Copano: Solo pienso en mi,9/8/2017,2017,TV-MA,"Fabrizio Copano takes audience participation to the next level in this stand-up set while reflecting on sperm banks, family WhatsApp groups and more.",80164077,null
null,1,TV Show,Fire Chasers,9/8/2017,2017,TV-MA,"As California's 2016 fire season rages, brave backcountry firefighters race to put out the flames, protect homes and save lives in this docuseries.",80117902,null
90,null,Movie,Good People,9/8/2017,2014,R,A struggling couple can't believe their luck when they find a stash of money in the apartment of a neighbor who was recently murdered.,70304990,null


In [0]:
df = df.fillna({"duration_minutes" : 0, "duration_seasons":0})

In [0]:
df = df.withColumn("duration_minutes",col('duration_minutes').cast(IntegerType()))\
        .withColumn("duration_seasons",col('duration_seasons').cast(IntegerType()))

In [0]:
df.printSchema()

root
 |-- duration_minutes: integer (nullable = true)
 |-- duration_seasons: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- description: string (nullable = true)
 |-- show_id: string (nullable = true)
 |-- _rescued_data: string (nullable = true)



In [0]:
df_vis = df.groupBy("type").agg(count("*").alias("total_count"))
display(df_vis)

type,total_count
null,1
TV Show,1969
Movie,4265
1944,1


Databricks visualization. Run in Databricks to view.

In [0]:
df_type_errors = df.filter(~df.type.isin('Movie', 'TV Show'))
df_type_errors.display()

duration_minutes,duration_seasons,type,title,date_added,release_year,rating,description,show_id,_rescued_data
null,null,1944,TV-PG,This documentary centers on the crew of the B-17 Flying Fortress Memphis Belle as it prepares to execute a strategic bombing mission over Germany.,80119194,null,null,null,null


In [0]:
df = df.where(df.type.isin('Movie', 'TV Show'))
df_vis = df.groupBy("type").agg(count("*").alias("total_count"))
display(df_vis)

type,total_count
TV Show,1969
Movie,4265


In [0]:
df = df.withColumn("short_title",split(col('title'),':')[0])
df.limit(10).display()

duration_minutes,duration_seasons,type,title,date_added,release_year,rating,description,show_id,_rescued_data,short_title
90,0,Movie,Norm of the North: King Sized Adventure,9/9/2019,2019,TV-PG,"Before planning an awesome wedding for his grandfather, a polar bear king must take back a stolen artifact from an evil archaeologist first.",81145628,null,Norm of the North
94,0,Movie,Jandino: Whatever it Takes,9/9/2016,2016,TV-MA,"""Jandino Asporaat riffs on the challenges of raising kids and serenades the audience with a rousing rendition of """"Sex on Fire"""" in his comedy show.""",80117401,null,Jandino
0,1,TV Show,Transformers Prime,9/8/2018,2013,TV-Y7-FV,"With the help of three human allies, the Autobots once again protect Earth from the onslaught of the Decepticons and their leader, Megatron.",70234439,null,Transformers Prime
0,1,TV Show,Transformers: Robots in Disguise,9/8/2018,2016,TV-Y7,"When a prison ship crash unleashes hundreds of Decepticons on Earth, Bumblebee leads a new Autobot force to protect humankind.",80058654,null,Transformers
99,0,Movie,#realityhigh,9/8/2017,2017,TV-14,"When nerdy high schooler Dani finally attracts the interest of her longtime crush, she lands in the cross hairs of his ex, a social media celebrity.",80125979,null,#realityhigh
0,1,TV Show,Apaches,9/8/2017,2016,TV-MA,A young journalist is forced into a life of crime to save his father and family in this series based on the novel by Miguel Sáez Carral.,80163890,null,Apaches
110,0,Movie,Automata,9/8/2017,2014,R,"In a dystopian future, an insurance adjuster for a tech company investigates a robot killed for violating protocol and discovers a global conspiracy.",70304989,null,Automata
60,0,Movie,Fabrizio Copano: Solo pienso en mi,9/8/2017,2017,TV-MA,"Fabrizio Copano takes audience participation to the next level in this stand-up set while reflecting on sperm banks, family WhatsApp groups and more.",80164077,null,Fabrizio Copano
0,1,TV Show,Fire Chasers,9/8/2017,2017,TV-MA,"As California's 2016 fire season rages, brave backcountry firefighters race to put out the flames, protect homes and save lives in this docuseries.",80117902,null,Fire Chasers
90,0,Movie,Good People,9/8/2017,2014,R,A struggling couple can't believe their luck when they find a stash of money in the apartment of a neighbor who was recently murdered.,70304990,null,Good People


In [0]:
df = df.withColumn("rating",split(col('rating'),'-')[1])\
        .fillna({'rating': 'R'})
df.limit(10).display()

duration_minutes,duration_seasons,type,title,date_added,release_year,rating,description,show_id,_rescued_data,short_title
90,0,Movie,Norm of the North: King Sized Adventure,9/9/2019,2019,PG,"Before planning an awesome wedding for his grandfather, a polar bear king must take back a stolen artifact from an evil archaeologist first.",81145628,null,Norm of the North
94,0,Movie,Jandino: Whatever it Takes,9/9/2016,2016,MA,"""Jandino Asporaat riffs on the challenges of raising kids and serenades the audience with a rousing rendition of """"Sex on Fire"""" in his comedy show.""",80117401,null,Jandino
0,1,TV Show,Transformers Prime,9/8/2018,2013,Y7,"With the help of three human allies, the Autobots once again protect Earth from the onslaught of the Decepticons and their leader, Megatron.",70234439,null,Transformers Prime
0,1,TV Show,Transformers: Robots in Disguise,9/8/2018,2016,Y7,"When a prison ship crash unleashes hundreds of Decepticons on Earth, Bumblebee leads a new Autobot force to protect humankind.",80058654,null,Transformers
99,0,Movie,#realityhigh,9/8/2017,2017,14,"When nerdy high schooler Dani finally attracts the interest of her longtime crush, she lands in the cross hairs of his ex, a social media celebrity.",80125979,null,#realityhigh
0,1,TV Show,Apaches,9/8/2017,2016,MA,A young journalist is forced into a life of crime to save his father and family in this series based on the novel by Miguel Sáez Carral.,80163890,null,Apaches
110,0,Movie,Automata,9/8/2017,2014,R,"In a dystopian future, an insurance adjuster for a tech company investigates a robot killed for violating protocol and discovers a global conspiracy.",70304989,null,Automata
60,0,Movie,Fabrizio Copano: Solo pienso en mi,9/8/2017,2017,MA,"Fabrizio Copano takes audience participation to the next level in this stand-up set while reflecting on sperm banks, family WhatsApp groups and more.",80164077,null,Fabrizio Copano
0,1,TV Show,Fire Chasers,9/8/2017,2017,MA,"As California's 2016 fire season rages, brave backcountry firefighters race to put out the flames, protect homes and save lives in this docuseries.",80117902,null,Fire Chasers
90,0,Movie,Good People,9/8/2017,2014,R,A struggling couple can't believe their luck when they find a stash of money in the apartment of a neighbor who was recently murdered.,70304990,null,Good People


In [0]:
df = df.withColumn("type_flag",when(col('type')=='Movie',1)\
                            .when(col('type')=='TV Show',2)\
                            .otherwise(0))
df.limit(10).display()

duration_minutes,duration_seasons,type,title,date_added,release_year,rating,description,show_id,_rescued_data,short_title,type_flag
90,0,Movie,Norm of the North: King Sized Adventure,9/9/2019,2019,PG,"Before planning an awesome wedding for his grandfather, a polar bear king must take back a stolen artifact from an evil archaeologist first.",81145628,null,Norm of the North,1
94,0,Movie,Jandino: Whatever it Takes,9/9/2016,2016,MA,"""Jandino Asporaat riffs on the challenges of raising kids and serenades the audience with a rousing rendition of """"Sex on Fire"""" in his comedy show.""",80117401,null,Jandino,1
0,1,TV Show,Transformers Prime,9/8/2018,2013,Y7,"With the help of three human allies, the Autobots once again protect Earth from the onslaught of the Decepticons and their leader, Megatron.",70234439,null,Transformers Prime,2
0,1,TV Show,Transformers: Robots in Disguise,9/8/2018,2016,Y7,"When a prison ship crash unleashes hundreds of Decepticons on Earth, Bumblebee leads a new Autobot force to protect humankind.",80058654,null,Transformers,2
99,0,Movie,#realityhigh,9/8/2017,2017,14,"When nerdy high schooler Dani finally attracts the interest of her longtime crush, she lands in the cross hairs of his ex, a social media celebrity.",80125979,null,#realityhigh,1
0,1,TV Show,Apaches,9/8/2017,2016,MA,A young journalist is forced into a life of crime to save his father and family in this series based on the novel by Miguel Sáez Carral.,80163890,null,Apaches,2
110,0,Movie,Automata,9/8/2017,2014,R,"In a dystopian future, an insurance adjuster for a tech company investigates a robot killed for violating protocol and discovers a global conspiracy.",70304989,null,Automata,1
60,0,Movie,Fabrizio Copano: Solo pienso en mi,9/8/2017,2017,MA,"Fabrizio Copano takes audience participation to the next level in this stand-up set while reflecting on sperm banks, family WhatsApp groups and more.",80164077,null,Fabrizio Copano,1
0,1,TV Show,Fire Chasers,9/8/2017,2017,MA,"As California's 2016 fire season rages, brave backcountry firefighters race to put out the flames, protect homes and save lives in this docuseries.",80117902,null,Fire Chasers,2
90,0,Movie,Good People,9/8/2017,2014,R,A struggling couple can't believe their luck when they find a stash of money in the apartment of a neighbor who was recently murdered.,70304990,null,Good People,1


In [0]:
df = df.withColumn("duration_ranking",dense_rank().over(Window.orderBy(col('duration_minutes').desc())))

In [0]:
df.limit(10).display()

duration_minutes,duration_seasons,type,title,date_added,release_year,rating,description,show_id,_rescued_data,short_title,type_flag,duration_ranking
312,0,Movie,Black Mirror: Bandersnatch,12/28/2018,2018,MA,"In 1984, a young programmer begins to question reality as he adapts a dark fantasy novel into a video game. A mind-bending tale with multiple endings.",80988062,null,Black Mirror,1,1
228,0,Movie,Sangam,12/31/2019,1964,14,"Returning home from war after being assumed dead, a pilot weds the woman he has long loved, unaware that she had been planning to marry his best friend.",60002818,null,Sangam,1,2
224,0,Movie,Lagaan,12/8/2017,2001,R,"In 1890s India, an arrogant British commander challenges the harshly taxed residents of Champaner to a high-stakes cricket match.",60020906,null,Lagaan,1,3
214,0,Movie,Jodhaa Akbar,10/1/2018,2008,14,"In 16th-century India, what begins as a strategic alliance between a Mughal emperor and a Hindu princess becomes a genuine opportunity for true love.",70090035,null,Jodhaa Akbar,1,4
209,0,Movie,The Irishman,11/27/2019,2019,R,Hit man Frank Sheeran looks back at the secrets he kept as a loyal member of the Bufalino crime family in this acclaimed film from Martin Scorsese.,80175798,null,The Irishman,1,5
205,0,Movie,The Gospel of Luke,10/19/2018,2015,14,Word-for-word Bible texts of the entire book of Luke are narrated and re-enacted in this epic production of the Gospel's accounts of Jesus's life.,81035749,null,The Gospel of Luke,1,6
203,0,Movie,What's Your Raashee?,8/15/2018,2009,PG,"To protect his family from ruin, Yogesh must marry his dream girl in only ten days, so he rushes into dating women with different astrological signs.",70123118,null,What's Your Raashee?,1,7
201,0,Movie,The Lord of the Rings: The Return of the King,1/1/2020,2003,13,"Aragorn is revealed as the heir to the ancient kings as he, Gandalf and the other members of the broken fellowship struggle to save Gondor.",60004484,null,The Lord of the Rings,1,8
200,0,Movie,Doctor Zhivago,11/1/2019,1965,13,A young physician and his beautiful mistress get swept up in the danger and drama of the Bolshevik Revolution in this Oscar-winning epic.,449931,null,Doctor Zhivago,1,9
196,0,Movie,Elephants Dream 4 Hour,8/23/2018,2006,MA,"Friends Proog and Emo live in a capricious, seemingly infinite machine with a sinister purpose in this experimental computer-animated short.",70274390,null,Elephants Dream 4 Hour,1,10


In [0]:
df.createOrReplaceTempView("temp_view") #If we want to create views

In [0]:
df.createOrReplaceGlobalTempView("global_view") #If we want to create a view for all notebooks

In [0]:
df = spark.sql("""
                select * 
                from global_temp.global_view
               """)

In [0]:
df.limit(10).display()

duration_minutes,duration_seasons,type,title,date_added,release_year,rating,description,show_id,_rescued_data,short_title,type_flag,duration_ranking
312,0,Movie,Black Mirror: Bandersnatch,12/28/2018,2018,MA,"In 1984, a young programmer begins to question reality as he adapts a dark fantasy novel into a video game. A mind-bending tale with multiple endings.",80988062,null,Black Mirror,1,1
228,0,Movie,Sangam,12/31/2019,1964,14,"Returning home from war after being assumed dead, a pilot weds the woman he has long loved, unaware that she had been planning to marry his best friend.",60002818,null,Sangam,1,2
224,0,Movie,Lagaan,12/8/2017,2001,R,"In 1890s India, an arrogant British commander challenges the harshly taxed residents of Champaner to a high-stakes cricket match.",60020906,null,Lagaan,1,3
214,0,Movie,Jodhaa Akbar,10/1/2018,2008,14,"In 16th-century India, what begins as a strategic alliance between a Mughal emperor and a Hindu princess becomes a genuine opportunity for true love.",70090035,null,Jodhaa Akbar,1,4
209,0,Movie,The Irishman,11/27/2019,2019,R,Hit man Frank Sheeran looks back at the secrets he kept as a loyal member of the Bufalino crime family in this acclaimed film from Martin Scorsese.,80175798,null,The Irishman,1,5
205,0,Movie,The Gospel of Luke,10/19/2018,2015,14,Word-for-word Bible texts of the entire book of Luke are narrated and re-enacted in this epic production of the Gospel's accounts of Jesus's life.,81035749,null,The Gospel of Luke,1,6
203,0,Movie,What's Your Raashee?,8/15/2018,2009,PG,"To protect his family from ruin, Yogesh must marry his dream girl in only ten days, so he rushes into dating women with different astrological signs.",70123118,null,What's Your Raashee?,1,7
201,0,Movie,The Lord of the Rings: The Return of the King,1/1/2020,2003,13,"Aragorn is revealed as the heir to the ancient kings as he, Gandalf and the other members of the broken fellowship struggle to save Gondor.",60004484,null,The Lord of the Rings,1,8
200,0,Movie,Doctor Zhivago,11/1/2019,1965,13,A young physician and his beautiful mistress get swept up in the danger and drama of the Bolshevik Revolution in this Oscar-winning epic.,449931,null,Doctor Zhivago,1,9
196,0,Movie,Elephants Dream 4 Hour,8/23/2018,2006,MA,"Friends Proog and Emo live in a capricious, seemingly infinite machine with a sinister purpose in this experimental computer-animated short.",70274390,null,Elephants Dream 4 Hour,1,10


In [0]:
df.write.format("delta")\
        .mode("overwrite")\
        .option("path","abfss://02-silver@project03storage.dfs.core.windows.net/netflix_titles")\
        .save()